In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Agg DF with SQL").getOrCreate()

In [2]:
file_path = "/content/utilization.json"
df = spark.read.format("json").load(file_path)

In [3]:
df.show(10)


+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [4]:
df.createOrReplaceTempView("utilization")

In [5]:
#When the data is not what we expect restart the session to refresh the variables
df_count = spark.sql("SELECT count (*) FROM utilization")
df_count.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [6]:
df_sql = spark.sql("SELECT count (*) \
                  FROM utilization \
                  WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [8]:
df_sql = spark.sql("SELECT server_id, count(*) \
                   FROM utilization \
                   WHERE session_count > 70 \
                   GROUP BY server_id")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      112|    7425|
|      113|    9418|
|      126|    6365|
|      110|    2826|
|      116|    1167|
|      107|    5646|
|      103|    8744|
|      114|    2128|
|      115|    5284|
|      104|    7366|
|      120|    2733|
|      111|    3093|
|      100|     391|
|      105|    1110|
|      108|    8375|
|      123|    7918|
|      101|    9808|
|      102|    8586|
|      118|    7913|
|      125|    2843|
+---------+--------+
only showing top 20 rows



In [9]:
df_sql = spark.sql("SELECT server_id, count(*) \
                   FROM utilization \
                   WHERE session_count > 70 \
                   GROUP BY server_id \
                   ORDER BY count(*) DESC ")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows



In [12]:
df_sql = spark.sql( "SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC"
                    )
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      